In [6]:
import json

with open('file/_sample_filtered_weibo_truth.txt', 'r') as src:
    lines = src.readlines()
    print(len(lines))
    for line in lines:
        event = json.loads(line)
        weibos = event['filtered_weibo']
        for weibo in weibos:
            print(weibo)
            
        break

4
{'userId': '5500586814', 'url': 'http://weibo.com/5500586814/DngkQ6So0', 'userLocation': '山东临沂', 'userCertify': 2, 'isOrigin': True, 'mid': '3955443401639032', 'sourcePlatform': '网易', 'praise': '0', 'content': '济宁学院大一男生因分手将女友捅死 两人均19岁 - 网易临沂 http://t.cn/RGD5mF5', 'containsMusic': False, 'userurl': 'http://weibo.com/u/5500586814?refer_flag=1001030103_', 'hotrate': 0.0, 'userFollowCount': '167', 'userWeiboCount': '11318', 'containsUrl': False, 'faces': None, 'isClue': False, 'forward': '0', 'userDescription': '网易临沂频道官方网址http://ly.news.163.com/', 'containsVideo': False, 'name': '网易新闻临沂站', 'time': '1458534658000', 'weiboLocation': None, 'piclist': ['http://ww3.sinaimg.cn/large/0060fTUajw1f24diedxytj30fa08caaj.jpg'], 'comment': '0', 'userGender': 'male', 'isPiyao': False, 'userFanCount': '482'}
{'userId': '2995514317', 'url': 'http://weibo.com/2995514317/DnhFVFznO', 'userLocation': '山东', 'userCertify': 1, 'isOrigin': True, 'mid': '3955494919907464', 'sourcePlatform': '百度分享', 'praise': '0'